In [ ]:
!git clone https://github.com/HoanChan/TableVision.git

In [3]:
def split_pth_file(input_file, output_file1, output_file2):
    with open(input_file, 'rb') as f:
        data = f.read()

    half_len = len(data) // 2

    with open(output_file1, 'wb') as f1, open(output_file2, 'wb') as f2:
        f1.write(data[:half_len])
        f2.write(data[half_len:])

def combine_pth_files(input_file1, input_file2, output_file):
    with open(input_file1, 'rb') as f1, open(input_file2, 'rb') as f2:
        data1 = f1.read()
        data2 = f2.read()

    combined_data = data1 + data2

    with open(output_file, 'wb') as f:
        f.write(combined_data)

In [4]:
# Sử dụng hàm để chia và kết hợp tệp .pth
split_pth_file("models\pubtables1m_detection_detr_r18.pth", "models\pubtables1m_detection_detr_r18.pth_part1", "models\pubtables1m_detection_detr_r18.pth_part2")
split_pth_file("models\pubtables1m_structure_detr_r18.pth", "models\pubtables1m_structure_detr_r18.pth_part1", "models\pubtables1m_structure_detr_r18.pth_part2")

In [ ]:
combine_pth_files("/content/TableVision/models/pubtables1m_detection_detr_r18.pth_part1", "/content/TableVision/models/pubtables1m_detection_detr_r18.pth_part2", "/content/TableVision/models/pubtables1m_detection_detr_r18.pth")
combine_pth_files("/content/TableVision/models/pubtables1m_structure_detr_r18.pth_part1", "/content/TableVision/models/pubtables1m_structure_detr_r18.pth_part2", "/content/TableVision/models/pubtables1m_structure_detr_r18.pth")


In [ ]:
!pip install PyMuPDF==1.21.1

In [ ]:
import sys
from PIL import Image

# Thêm đường dẫn đến thư mục 'src' vào danh sách đường dẫn
sys.path.append('/content/TableVision/')
sys.path.append('/content/TableVision/src/')
sys.path.append('/content/TableVision/detr/')
from src.inference import TableExtractionPipeline

# Create inference pipeline
pipe = TableExtractionPipeline(det_config_path='/content/TableVision/src/detection_config.json', 
                               det_model_path='/content/TableVision/models/pubtables1m_detection_detr_r18.pth', det_device='cuda', 
                               str_config_path='/content/TableVision/src/structure_config.json', 
                               str_model_path='/content/TableVision/models/pubtables1m_structure_detr_r18.pth', str_device='cuda')

# Recognize table(s) from image

img_path = "/content/TableVision/sample/tablex2.png"
img = Image.open(img_path)
tokens = {}
extracted_tables = pipe.recognize(img, tokens, out_objects=True, out_cells=True, out_html=True, out_csv=True)

# Select table (there could be more than one)
try:
  extracted_table = extracted_tables[0]
except:
  extracted_table = extracted_tables

# Get output in desired format
objects = extracted_table['objects']
cells = extracted_table['cells']
csv = extracted_table['csv']
html = extracted_table['html']

